# **Bank transactions**
## Data preparation

Carlos Garavito
***

In [58]:
# import packages
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

# set no limit to raw display with pandas
pd.set_option('display.max_rows', None)

# define working paths
PROJECT_PATH = os.getcwd()
RAWDATA_PATH = os.path.join(PROJECT_PATH, os.pardir, 'data', 'raw')
INTERIM_PATH = os.path.join(PROJECT_PATH, os.pardir, 'data', 'interim')
PROCDATA_PATH = os.path.join(PROJECT_PATH, os.pardir, 'data', 'processed')

In [59]:
# read interim data
file_1 = os.path.join(INTERIM_PATH, "bank_campaign_v2.csv")
df_int_data = pd.read_csv(file_1)

In [60]:
df_int_data.sample(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,cons.conf.idx,euribor3m,subscribed
38735,72,retired,married,professional.course,no,yes,no,telephone,nov,wed,93,2,999,0,nonexistent,-30.1,0.715,no
6061,37,blue-collar,single,professional.course,no,yes,no,telephone,may,tue,213,2,999,0,nonexistent,-36.4,4.857,no
7587,30,blue-collar,married,basic.6y,no,no,no,telephone,may,fri,88,2,999,0,nonexistent,-36.4,4.864,no
37330,58,technician,married,professional.course,no,no,no,cellular,aug,tue,704,2,999,0,nonexistent,-31.4,0.883,yes
3777,33,blue-collar,married,professional.course,no,yes,no,telephone,may,fri,208,2,999,0,nonexistent,-36.4,4.859,no


## Imputing null values
***

There are no null values. This was checked in the [EDA notebook](#).

In [61]:
# identify null values by percentage
missing_values = df_int_data.isnull()
missing_values.sum() / len(df_int_data)

age              0.0
job              0.0
marital          0.0
education        0.0
default          0.0
housing          0.0
loan             0.0
contact          0.0
month            0.0
day_of_week      0.0
duration         0.0
campaign         0.0
pdays            0.0
previous         0.0
poutcome         0.0
cons.conf.idx    0.0
euribor3m        0.0
subscribed       0.0
dtype: float64

## Imputing duplicated values
***

In [62]:
print(f"Data set has {len(df_int_data[df_int_data.duplicated(keep='last')])} duplicated values")

Data set has 12 duplicated values


In [63]:
# check for duplicated rows
df_int_data[df_int_data.duplicated(keep='last')]

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,cons.conf.idx,euribor3m,subscribed
1265,39,blue-collar,married,basic.6y,no,no,no,telephone,may,thu,124,1,999,0,nonexistent,-36.4,4.855,no
12260,36,retired,married,unknown,no,no,no,telephone,jul,thu,88,1,999,0,nonexistent,-42.7,4.966,no
14155,27,technician,single,professional.course,no,no,no,cellular,jul,mon,331,2,999,0,nonexistent,-42.7,4.962,no
16819,47,technician,divorced,high.school,no,yes,no,cellular,jul,thu,43,3,999,0,nonexistent,-42.7,4.962,no
18464,32,technician,single,professional.course,no,yes,no,cellular,jul,thu,128,1,999,0,nonexistent,-42.7,4.968,no
20072,55,services,married,high.school,unknown,no,no,cellular,aug,mon,33,1,999,0,nonexistent,-36.1,4.965,no
20531,41,technician,married,professional.course,no,yes,no,cellular,aug,tue,127,1,999,0,nonexistent,-36.1,4.966,no
25183,39,admin.,married,university.degree,no,no,no,cellular,nov,tue,123,2,999,0,nonexistent,-42.0,4.153,no
28476,24,services,single,high.school,no,yes,no,cellular,apr,tue,114,1,999,0,nonexistent,-47.1,1.423,no
32505,35,admin.,married,university.degree,no,yes,no,cellular,may,fri,348,4,999,0,nonexistent,-46.2,1.313,no


In [64]:
# drop duplicated rows
df_int_data = df_int_data.drop_duplicates(keep='last')

In [65]:
print(f"Data set after duplicated values imputation, have {df_int_data.shape[0]} rows and {df_int_data.shape[1]} columns.")

Data set after duplicated values imputation, have 41173 rows and 18 columns.


## Separating variables into features and target
***

In [66]:
target = df_int_data.subscribed
df_features = df_int_data.drop(columns=["subscribed"])

In [67]:
target.sample(5)

15021    no
8508     no
10898    no
12149    no
20861    no
Name: subscribed, dtype: object

In [68]:
df_features.sample(5)

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,cons.conf.idx,euribor3m
40816,30,technician,single,university.degree,no,yes,no,cellular,sep,fri,173,2,15,4,failure,-37.5,0.879
14959,38,management,single,university.degree,no,no,no,telephone,jul,wed,284,8,999,0,nonexistent,-42.7,4.957
29540,50,services,married,high.school,no,yes,no,cellular,apr,mon,12,6,999,0,nonexistent,-47.1,1.405
3189,34,admin.,single,high.school,no,yes,no,telephone,may,thu,168,5,999,0,nonexistent,-36.4,4.860
7150,50,technician,divorced,basic.9y,no,no,yes,telephone,may,thu,48,5,999,0,nonexistent,-36.4,4.860


## Data preparation 
***

In [69]:
# read data
y = target
X = df_features

In [70]:
# replace 'yes' and 'no' values with 0 and 1
y.replace({"no":0, "yes":1}, inplace=True)
y.head(5)

0    0
1    0
2    0
3    0
4    0
Name: subscribed, dtype: int64

**Asumption 1** To avoid different dimensionality due one get_dummies method due different values on samples, one hoy encoding its performed over the whole data set, and, later the slipt will be performed.

In [71]:
# one hot encoding values 
X  =  pd.get_dummies(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41173 entries, 0 to 41184
Data columns (total 60 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   age                            41173 non-null  int64  
 1   duration                       41173 non-null  int64  
 2   campaign                       41173 non-null  int64  
 3   pdays                          41173 non-null  int64  
 4   previous                       41173 non-null  int64  
 5   cons.conf.idx                  41173 non-null  float64
 6   euribor3m                      41173 non-null  float64
 7   job_admin.                     41173 non-null  uint8  
 8   job_blue-collar                41173 non-null  uint8  
 9   job_entrepreneur               41173 non-null  uint8  
 10  job_housemaid                  41173 non-null  uint8  
 11  job_management                 41173 non-null  uint8  
 12  job_retired                    41173 non-null 

In [72]:
# check first register - without encoding
df_features.head(1).T

,0
age,56
job,housemaid
marital,married
education,basic.4y
default,no
housing,no
loan,no
contact,telephone
month,may
day_of_week,mon


In [73]:
# check first register - one hot encoding
X.head(1).T

,0
age,56.000
duration,261.000
campaign,1.000
pdays,999.000
previous,0.000
cons.conf.idx,-36.400
euribor3m,4.857
job_admin.,0.000
job_blue-collar,0.000
job_entrepreneur,0.000


## Split data into train, test and validation

In [74]:
# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

# split into train and validation
X_train, X_CV, y_train, y_CV = train_test_split(X_train, y_train, test_size=0.3, random_state=101)

print(f"Train data set contains {len(X_train)} samples")
print(f"Test data set contains {len(X_test)} samples")
print(f"Validation data set contains {len(X_CV)} samples")
print("---")
print(f"Total samples: {len(X_train) +  len(X_test) + len(X_CV)}  ")


Train data set contains 20174 samples
Test data set contains 12352 samples
Validation data set contains 8647 samples
---
Total samples: 41173  


## Save preparated data
***

In [76]:
# save train data
X_train.to_csv(os.path.join(PROCDATA_PATH, 'train', 'X_train.csv'), index=False)
y_train.to_csv(os.path.join(PROCDATA_PATH, 'train', 'y_train.csv'), index=False)

# save test data
X_test.to_csv(os.path.join(PROCDATA_PATH, 'test', 'X_test.csv'), index=False)
y_test.to_csv(os.path.join(PROCDATA_PATH, 'test', 'y_test.csv'), index=False)

# save validation data
X_CV.to_csv(os.path.join(PROCDATA_PATH, 'validation', 'X_CV.csv'), index=False)
y_CV.to_csv(os.path.join(PROCDATA_PATH, 'validation', 'y_CV.csv'), index=False)